In [10]:
from datasets import load_happines
from tools import *
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import plotly.express as px
import pandas as pd
from sklearn.model_selection import cross_val_score

In [11]:
df_train, df_test = load_happines()
df_train.columns

Index(['region', 'economy', 'family', 'health', 'freedom', 'trust',
       'generosity', 'target'],
      dtype='object')

In [12]:
train_X = df_train.drop(['region', 'target'], axis=1)
train_y = df_train['target']
test_X = df_test.drop(['region', 'target'], axis=1)
test_y = df_test['target']
features = train_X.columns

In [13]:
scaler = StandardScaler().fit(train_X)
train_X = pd.DataFrame(scaler.transform(train_X), columns=features)
test_X = pd.DataFrame(scaler.transform(test_X), columns=features)

## Pocet komponent

In [29]:
pca = PCA(n_components=6)
pca = pca.fit(train_X)

In [30]:
# vidime ze nejvice variance popisuji prvni 3-4 komponenty, zvolime 3 pro lepsi vizualizaci
pca.explained_variance_ratio_

array([0.47947993, 0.22077254, 0.11718491, 0.09331852, 0.06422008,
       0.02502402])

## Regresion

In [31]:
pca = PCA(n_components=3)
pca = pca.fit(train_X)

In [32]:
transformed_train_X = pca.transform(train_X)
transformed_test_X = pca.transform(test_X)

In [33]:
transformed_df = pd.DataFrame(transformed_train_X, columns=['x','y','z'])
transformed_df['target'] = train_y.values
px.scatter_3d(transformed_df, x='x', y='y', z='z',
              color='target', size='target')

In [34]:
cross_val_score(LinearRegression(), transformed_train_X, train_y).mean()

0.7463578101159014

In [35]:
model = LinearRegression()
model = model.fit(transformed_train_X, train_y)

In [36]:
model_performance(model, transformed_train_X, train_y, transformed_test_X, test_y)

Train score
0.7724388212760959
Test score
0.7110815512648236


## Overeni vyberu komponent

In [24]:
scores = []
components = []
for i in range(1, len(df_train.columns) - 1):
    pca = PCA(n_components=i)
    pca = pca.fit(train_X)
    transformed_train_X = pca.transform(train_X)
    transformed_test_X = pca.transform(test_X)
    components.append(i)
    scores.append(cross_val_score(LinearRegression(), transformed_train_X, train_y).mean())

pd.DataFrame(zip(components, scores), columns=['components', 'score'])

,components,score
0,1,0.734978
1,2,0.750945
2,3,0.746358
3,4,0.744375
4,5,0.743989
5,6,0.742787
